<a href="https://colab.research.google.com/github/mwtam/blog/blob/main/webcmp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
def all_work():
    work = {
        "example": [
            "https://example.com",
            ".*"
        ],
    }
    return work


def new_work():
    work = {
        "example": [
            "https://exampleaaa.com",
            ".*"
        ],
    }

    return work

In [2]:
import re
import requests
from datetime import datetime
import enum


class VisitResult(enum.IntEnum):
    CHANGED = enum.auto()
    UNCHANGED = enum.auto()
    NEW_TITLE = enum.auto()
    FAILED = enum.auto()
    UNKNOWN_ERROR = enum.auto()


def load_page(url):
    try:
        res = requests.get(url, headers={'User-Agent': 'Custom'})
        return res.status_code, res.text
    except requests.exceptions.ConnectionError as e:
        return e, ""


def visit(title, url, re_str):
    rc, webpage = load_page(url)
    if rc != requests.codes.ok:
        return VisitResult.FAILED, f"Fail to get the page: {rc}"

    p = re.compile(re_str)
    result = set(l.strip() for l in p.findall(webpage))

    with open(f"{title}_new.txt", "w") as f:
        f.write("\n".join(result))

    try:
        # Compare with the old result
        with open(f"{title}.txt") as f:
            old_result = set([l.strip() for l in f])
    except FileNotFoundError:
        with open(f"{title}.txt", "w") as f:
            f.write("\n".join(result))
        return VisitResult.NEW_TITLE, "New Title"

    if result == old_result:
        return VisitResult.UNCHANGED, "Nothing Changed"
    else:
        return VisitResult.CHANGED, "Updated"


def main():
    work = all_work()
    # work = new_work()

    results = []
    for title, (url, re_str) in work.items():
        print(f"Working on {title}")
        # print(f"{url=}")
        # print(f"{re_str}")
        try:
            result, msg = visit(title, url, re_str)
            results.append((result, title, url, msg))
        except Exception as e:
            results.append((VisitResult.UNKNOWN_ERROR, title, url, str(e)))

    results.sort()
    for t in results:
        print(t)

In [7]:
main()

Working on example
(<VisitResult.UNCHANGED: 2>, 'example', 'https://example.com', 'Nothing Changed')
